In [1]:
import cv2 as cv
import numpy as np

In [2]:
def initAlgorithm() -> None:
    cap = cv.VideoCapture('16.2.mp4')
    ret, old_frame = cap.read()
    old_gray = cv.cvtColor(old_frame, cv.COLOR_BGR2GRAY)
    p0 = cv.goodFeaturesToTrack(old_gray, mask = None, **feature_params)
    mask = np.zeros_like(old_frame)
    
    return cap, old_gray, p0, mask

In [3]:
# params for ShiTomasi corner detection
feature_params = dict( maxCorners = 100,
                       qualityLevel = 0.3,
                       minDistance = 7,
                       blockSize = 7 )

# Parameters for lucas kanade optical flow
lk_params = dict( winSize  = (15, 15),
                  maxLevel = 2,
                  criteria = (cv.TERM_CRITERIA_EPS | cv.TERM_CRITERIA_COUNT, 10, 0.03))

# Create some random colors
color = np.random.randint(0,255, (100, 3))

# Take first frame and find corners in it
cap, old_gray, p0, mask = initAlgorithm()

while True:
    ret, frame = cap.read()
    
    if ret:
        # frame = cv.resize(frame, new_size)
        frame_gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
        
        # calculate optical flow
        p1, st, err = cv.calcOpticalFlowPyrLK(old_gray, frame_gray, p0, None, **lk_params)
        
        # Select good points
        if p1 is not None:
            good_new = p1[st == 1]
            good_old = p0[st == 1]
        
        # draw the tracks
        for i,(new, old) in enumerate(zip(good_new, good_old)):
            a, b = new.ravel()
            c, d = old.ravel()
            mask = cv.line(mask, (int(a), int(b)),(int(c), int(d)), color[i].tolist(), 2)
            frame = cv.circle(frame,(int(a), int(b)), 5, color[i].tolist(),-1)
            
        img = cv.add(frame, mask)
        cv.imshow('frame',img)
        
        if cv.waitKey(20) & 0xFF == ord('q'):
            break
        
        # Now update the previous frame and previous points
        old_gray = frame_gray.copy()
        p0 = good_new.reshape(-1, 1, 2)
    else:
        cap.release()
        cv.destroyAllWindows()
        cap, old_gray, p0, mask = initAlgorithm()

cap.release()
cv.destroyAllWindows()